In [8]:
import ee
import geopandas as gpd
from shapely.geometry import mapping
import pandas as pd

# Authenticate and Initialize EE (only need to authenticate once)
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

# Load zones from a GeoJSON or Shapefile
gdf = gpd.read_file(r"D:\Projects\TMU\gee_dengue\data\external\sa\Bantustan_shpfile.geojson")  # Replace with your shapefile if needed
features = []

# Convert GeoPandas geometries to EE Features
for _, row in gdf.iterrows():
    geom = ee.Geometry(mapping(row['geometry']))
    feature = ee.Feature(geom, row.drop('geometry').to_dict())
    features.append(feature)

# EE FeatureCollection
fc = ee.FeatureCollection(features)

# Prepare output list
all_stats = []

# Loop over each year
for year in range(2001, 2021):  # Inclusive of 2020
    print(f"Processing year: {year}")

    # Load image for the year
    image = ee.ImageCollection("MODIS/061/MCD12Q1") \
                .filterDate(f"{year}-01-01", f"{year}-12-31") \
                .first() \
                .select("LC_Type1")

    # Zonal stats function
    def compute_stats(feature):
        stats = image.reduceRegion(
            reducer=ee.Reducer.frequencyHistogram(),
            geometry=feature.geometry(),
            scale=500,
            maxPixels=1e13
        )
        return feature.set({'year': year, 'landcover_hist': stats.get('LC_Type1')})

    # Apply to all features
    result = fc.map(compute_stats).getInfo()

    # Flatten for pandas
    for feat in result['features']:
        props = feat['properties']
        lc_hist = props.get('landcover_hist', {})
        props.update(lc_hist)  # Flatten histogram
        props.pop('landcover_hist', None)
        props['year'] = year
        all_stats.append(props)

# Convert to DataFrame
df = pd.DataFrame(all_stats)

# Save to CSV
df.to_csv("modis_landcover_zonal_stats_2001_2020.csv", index=False)

print("✅ Export complete: modis_landcover_zonal_stats_2001_2020.csv")


Processing year: 2001
Processing year: 2002
Processing year: 2003
Processing year: 2004
Processing year: 2005
Processing year: 2006
Processing year: 2007
Processing year: 2008
Processing year: 2009
Processing year: 2010
Processing year: 2011
Processing year: 2012
Processing year: 2013
Processing year: 2014
Processing year: 2015
Processing year: 2016
Processing year: 2017
Processing year: 2018
Processing year: 2019
Processing year: 2020
✅ Export complete: modis_landcover_zonal_stats_2001_2020.csv
